## Model Analysis w session OHE info

In [2]:
#import sys
#sys.path.insert(0,"/Library/Python/2.7/site-packages/") 
#sys.path.insert(0,"/Library/Frameworks/Python.framework/Versions/2.7/bin")
%load_ext autoreload
%autoreload 1 # autoreload all modules imported w %aimport automatically before code execution

import pandas as pd
import numpy as np
from sklearn.cross_validation import train_test_split
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cross_validation import cross_val_score
from sklearn import metrics

In [3]:
data = pd.read_csv("./data/df_all_inc_sess.csv")

In [ ]:
data.head()

In [4]:
# get X and y
X = data[0:213466]
y = pd.read_csv("./data/y.csv")

In [5]:
# drop last 15 samples as they're caused by a bug (https://www.kaggle.com/c/airbnb-recruiting-new-user-bookings/forums/t/17737/announcement-competition-will-restart-shortly)
X = X[0:213451]
y = y[0:213451]

In [6]:
# take only samples w session info
X = X[X.hit_count > 0]
y = y.ix[X.index]

In [7]:
# drop user_id
X = X.drop(['user_id'], axis=1)

In [8]:
# for users w/out session info, set relevant features to -1.
X.fillna(value=-1, inplace=True)

In [16]:
print X.shape
print y.shape

(73815, 770)
(73815,)


In [10]:
# encode y values
y_labels = {}
i = 0
for value in y.country_destination.unique():
    if value not in y_labels.keys():
        y_labels[value] = i
        i += 1
    y[y == value] = i-1

print "y is indexed by dict:\n%s" % y_labels

y is indexed by dict:
{'FR': 4, 'NL': 10, 'PT': 11, 'CA': 6, 'DE': 8, 'IT': 5, 'US': 3, 'NDF': 1, 'other': 0, 'AU': 7, 'GB': 2, 'ES': 9}


In [11]:
# set y to array-like type
y = y.country_destination
y.ravel().astype(int)

array([0, 1, 1, ..., 1, 1, 1])

In [12]:
# train / test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3, random_state=1)

In [32]:
print X_train.shape
print y_train.shape
print X_test.shape
print y_test.shape

(51670, 770)
(51670,)
(22145, 770)
(22145,)


In [18]:
X_train.to_csv("./data/temp/X_train.csv", index=False)
y_train.to_csv("./data/temp/y_train.csv", index=False)

In [31]:
X_test.to_csv("./data/temp/X_test.csv", index=False)
y_test.to_csv("./data/temp/y_test.csv", index=False)

In [15]:
X_100 = X_train[0:100]
y_100 = y_train[0:100]
X_100.to_csv("./data/temp/X_100.csv", index=False)
y_100.to_csv("./data/temp/y_100.csv", index=False)

## Evio.evolver.ConceptEvolutionClassifier

In [ ]:
import sys
sys.path.insert(0, "/Users/gokhan/libs")
%aimport from Evio.evolver import ConceptEvolutionClassifier

In [ ]:
cec = ConceptEvolutionClassifier(n_concepts=2, random_state=1)

In [ ]:
cec.p_mut


In [ ]:
cec.fit(X_train[0:100], y_train[0:100], 12, n_folds=1)

## AdaBoost accuracy 0.6371 (n_estimators=200) (NA values filled with -1)
## n_estimators=100 => acc. 0.6352 (NA values filled with -1)
## n_estimators=300 => acc. 0.6357 (NA values filled with -1)


In [ ]:
# adaboost
from sklearn.ensemble import AdaBoostClassifier

abo = AdaBoostClassifier(n_estimators=300)
abo.fit(X_train, y_train.astype(int))
y_pred_abo = abo.predict(X_test)

In [ ]:
print metrics.accuracy_score(y_test.astype(int), y_pred_abo.astype(int))

In [ ]:
conf_matrix_abo = metrics.confusion_matrix(y_test.astype(int), y_pred_abo.astype(int))
print conf_matrix_abo
print sum(sum(conf_matrix_abo))

In [ ]:
predict_proba_test_abo = abo.predict_proba(X_test)
predict_proba_test_abo.shape

In [ ]:
predict_proba_test_abo[:3]

In [ ]:
feat_importances = []
for i in range(len(X_test.columns)):
    #print "%s: %.3f" % (X_test.columns[i], abo.feature_importances_[i])
    feat_importances.append((X_test.columns[i], abo.feature_importances_[i]))

feat_importances = sorted(feat_importances, key=lambda x: x[1], reverse=True)
for pair in feat_importances:
    print pair[0], pair[1]


#print X_train.columns
#print abo.feature_importances_

## Gaussian Naive Bayes Classifier
## Accuracy 0.5655

In [ ]:
# Naive Bayes
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
y_pred_gnb = gnb.fit(X_train, y_train.astype(int)).predict(X_test)

In [ ]:
print metrics.accuracy_score(y_test.astype(int), y_pred_gnb.astype(int))

In [ ]:
conf_matrix_gnb = metrics.confusion_matrix(y_test.astype(int), y_pred_gnb.astype(int))
print conf_matrix_gnb
print sum(sum(conf_matrix_gnb))

## DecisionTree Classifier
## Acc: 0.5250

In [ ]:
from sklearn import tree
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X_train, y_train.astype(int))
y_pred_dtc = clf.predict(X_test)

In [ ]:
print metrics.accuracy_score(y_test.astype(int), y_pred_dtc.astype(int))

In [ ]:
conf_matrix_dtc = metrics.confusion_matrix(y_test.astype(int), y_pred_dtc.astype(int))
print conf_matrix_dtc
print sum(sum(conf_matrix_dtc))

## RandomForest Classifier
## n_estimators=10   ; Acc= 0.6054
## n_estimators=100 ; Acc= 0.6201
## n_estimators=300 ; Acc= 0.6214

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_estimators=300, n_jobs=4, verbose=True)
rfc = rfc.fit(X_train, y_train.astype(int))
y_pred_rfc = rfc.predict(X_test)

In [ ]:
print metrics.accuracy_score(y_test.astype(int), y_pred_rfc.astype(int))

In [ ]:
conf_matrix_rfc = metrics.confusion_matrix(y_test.astype(int), y_pred_rfc.astype(int))
print conf_matrix_rfc
print sum(sum(conf_matrix_rfc))

## eXtremeGradientBoosting
## Acc=0.6383 (max_depth=2, eta=1, multi:softmax, ) => w a eval-merror: ~0.361665
## Acc=0.42 !!!! (max_depth=2, eta=1, multi:softmax, missing=-1) => w a eval-meror: ~0.361430
## Acc=0.5755 (max_depth=4, eta=1, multi:softmax, missing=-1) => w a eval-meror: ~0.354794
## Acc=0.58 (max_depth=4, eta=1, multi:softmax, missing=-1) => w a eval-meror: ~0.352108
## Acc=0.6271 (max_depth=4, eta=1, multi:softmax, missing=-1) => w a eval-meror: ~0.358729
## Acc=0.6273(max_depth=6, eta=0.1, multi:softmax, missing=-1) => w a(eval-merror:0.350515	train-merror:0.346258)
## Acc=0.6299(max_depth=6, eta=0.1, multi:softmax, missing=-1) => w ([16]	eval-merror:0.351156	train-merror:0.347570)
## Acc=0.6309(max_depth=6, eta=0.1, multi:softmax, missing=-1) => w ([1]	eval-merror:0.354700	train-merror:0.353419)
## Acc=0.6272(max_depth=6, eta=0.1, multi:softmax, missing=-1, eval_metric=auc) => w ([24]	eval-auc:0.530073	train-auc:0.498108)
## Acc=0.6281(max_depth=6, eta=0.1, multi:softmax, missing=-1, eval_metric=auc) => w ([19]	eval-auc:0.530904	train-auc:0.497756)
## Acc=0.6284(max_depth=6, eta=0.1, multi:softmax, missing=-1, eval_metric=auc) => w ([20]	eval-auc:0.530723	train-auc:0.498060)

## Acc=0.6170 (max_depth=2, eta=0.1, multi:softmax, eval_metric=auc) => w ([24]	eval-auc:0.530798	train-auc:0.498628)
## Acc=0.6366 (max_depth=2, eta=0.1, multi:softmax, eval_metric=auc) => w ([10]	eval-auc:0.530994	train-auc:0.499466)
## Acc=0.6388 (max_depth=2, eta=0.1, multi:softmax, eval_metric=auc) => w ([5]	eval-auc:0.530022	train-auc:0.500733)
## Acc=0.6391 (max_depth=2, eta=0.1, multi:softmax, eval_metric=auc) => w ([7]	eval-auc:0.531386	train-auc:0.500006)
## Acc=0.6253 (max_depth=2, eta=0.1, multi:softmax, eval_metric=auc) => w ([49]	eval-auc:0.531864	train-auc:0.500586)
## Acc=0.6246 (max_depth=2, eta=0.1, multi:softmax, eval_metric=auc) => w ([48]	eval-auc:0.531936	train-auc:0.500466)
## Acc=0.6208 (max_depth=2, eta=0.1, multi:softmax, eval_metric=auc) => w ([45]	eval-auc:0.531965	train-auc:0.500355)

## Acc=0.6380 (max_depth=3, eta=0.1, multi:softmax, eval_metric=auc) => w ([49]	eval-auc:0.532793	train-auc:0.500927)
## Acc=0.6369 (max_depth=3, eta=0.1, multi:softmax, eval_metric=auc) => w ([42]	eval-auc:0.533173	train-auc:0.500538)
## Acc=0.6377 (max_depth=3, eta=0.1, multi:softmax, eval_metric=auc) => w ([48]	eval-auc:0.532941	train-auc:0.500901)
## Acc=0.6406 (max_depth=3, eta=0.1, multi:softmax, eval_metric=auc) => w ([99]	eval-auc:0.532941	train-auc:0.500901)
## Acc=0.6398 (max_depth=3, eta=0.1, multi:softmax, eval_metric=auc) => w ([50+28]	eval-auc:0.532336	train-auc:0.501438)
## Acc=0.6404 (max_depth=3, eta=0.1, multi:softmax, eval_metric=auc) => w ([150] eval-auc:0.531397	train-auc:0.500827)
## Acc=0.6403 (max_depth=3, eta=0.1, multi:softmax, eval_metric=auc) => w ([100]	eval-auc:0.532247	train-auc:0.501267)
## Acc=0.6404 (max_depth=3, eta=0.1, multi:softmax, eval_metric=auc) => w ([102] eval-auc:0.532046	train-auc:0.501307)

## Acc=0.6307 (max_depth=4, eta=0.1, multi:softmax, eval_metric=auc) => w ([199]	eval-auc:0.530604	train-auc:0.500575)
## Acc=0.6286 (max_depth=4, eta=0.1, multi:softmax, eval_metric=auc) => w ([0]	eval-auc:0.536492	train-auc:0.499501)
## Acc=0.6362 (max_depth=4, eta=0.1, multi:softmax, eval_metric=auc) => w ([30]	eval-auc:0.532703	train-auc:0.499100)
## Acc=0.6341 (max_depth=4, eta=0.1, multi:softmax, eval_metric=auc) => w ([42]	eval-auc:0.532502	train-auc:0.500677)
## Acc=0.6333 (max_depth=4, eta=0.1, multi:softmax, eval_metric=auc) => w ([86]	eval-auc:0.532818	train-auc:0.501482)
## Acc=0.6332 (max_depth=4, eta=0.1, multi:softmax, eval_metric=auc) => w ([140]	eval-auc:0.530992	train-auc:0.501007)


In [ ]:
 import xgboost as xgb

#param = {'bst:max_depth':2, 'bst:eta':1, 'silent':1, 'objective':'binary:logistic' }
param = {'bst:max_depth':4, 'bst:eta':0.1, 'silent':0, 'objective': 'multi:softmax', 'num_class': 12}
param['nthread'] = 4
param['eval_metric'] = 'auc'

num_round = 200
dtrain = xgb.DMatrix( X_train, label=y_train, missing=-1)
dtest = xgb.DMatrix( X_test, label=y_test, missing=-1)
evallist  = [(dtest,'eval'), (dtrain,'train')]
#bst = xgb.train( param.items(), dtrain, num_round, evallist, early_stopping_rounds=10)
bst = xgb.train( param.items(), dtrain, num_round, evallist)

In [ ]:
# continue previous training
bst = xgb.train( param.items(), dtrain, num_round, evallist, xgb_model=bst)

In [ ]:
print bst.best_ntree_limit
print bst.best_iteration
#print bst.best_score

In [ ]:
# We must set ntree_limit=bst.best_ntree_limit, 
# and prior to that bst.best_ntree_limit must be set to: [best_eval-merror_iteration_index+1]
bst.best_ntree_limit = 141
bst.best_iteration = 140

In [ ]:
y_pred_xgb = bst.predict(xgb.DMatrix(X_test), ntree_limit=bst.best_ntree_limit)

In [ ]:
print metrics.accuracy_score(y_test.astype(int), y_pred_xgb.astype(int))

In [ ]:
conf_matrix_xgb = metrics.confusion_matrix(y_test.astype(int), y_pred_xgb.astype(int))
print conf_matrix_xgb
print sum(sum(conf_matrix_xgb))

## KNN

In [ ]:
knn = KNeighborsClassifier(n_neighbors=2, weights='distance')
knn.fit(X_train, y_train.astype(int))
y_pred_knn = knn.predict(X_test)

In [ ]:
conf_matrix_knn = metrics.confusion_matrix(y_test.astype(int), y_pred_knn.astype(int))
print conf_matrix_knn

print "Total items: %d" % sum(sum(conf_matrix_knn))

print "Accuracy: %.4f" % metrics.accuracy_score(y_test.astype(int), y_pred_knn.astype(int))

## SVM takes too much time!!!


In [ ]:
from sklearn.svm import SVC

svm = SVC(C=1.0, verbose=True)
svm.fit(X_train, y_train.astype(int))
y_pred_svm = svm.predict(X_test)

In [ ]:
print metrics.accuracy_score(y_test.astype(int), y_pred_svm.astype(int))

In [ ]:
conf_matrix_svm = metrics.confusion_matrix(y_test.astype(int), y_pred_svm.astype(int))
print conf_matrix_svm
print sum(sum(conf_matrix_svm))

### GradientBoosting (n_estimators=100, max_depth=3, learning_rate=0.1) => Acc=0.64815 !!!! 
### GradientBoosting (n_estimators=25, max_depth=3, learning_rate=0.1) => Acc=0.6495 !!!!
## GradientBoosting (n_estimators=50, max_depth=3, learning_rate=0.1) => Acc=0.6506 !!!! [BEST !!!] 
### GradientBoosting (n_estimators=200, max_depth=3, learning_rate=0.1) => Acc= 0.6477 (takes ~10 hrs!)

### GradientBoosting (n_estimators=50, max_depth=3, learning_rate=0.1) => Acc=0.6506 (sess w 1,2,4,... hrs) - NO CHANGE

### GradientBoosting (n_estimators=50, max_depth=3, learning_rate=0.1) => Acc=0.7010 (training set is comprised ONLY of samples w session info) 
### GradientBoosting (n_estimators=100, max_depth=3, learning_rate=0.1) => Acc=0.6996 (training set is comprised ONLY of samples w session info) 

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

gbc = GradientBoostingClassifier(n_estimators=100, verbose=1)
gbc.fit(X_train, y_train.astype(int))
y_pred_gbc = gbc.predict(X_test)

In [ ]:
print metrics.accuracy_score(y_test.astype(int), y_pred_gbc.astype(int))

In [ ]:
conf_matrix_gbc = metrics.confusion_matrix(y_test.astype(int), y_pred_gbc.astype(int))
print conf_matrix_gbc
print sum(sum(conf_matrix_gbc))

In [ ]:
predict_proba_test_gbc = gbc.predict_proba(X_test)
predict_proba_test_gbc.shape

In [ ]:
print y_labels

In [ ]:
y_indexes = {}
for country,index in y_labels.items():
    y_indexes[index] = country
print y_indexes

In [ ]:
for i in range(5):
    probas = predict_proba_test_gbc[i]
    probas_indexed = []
    for j in range(12):
        probas_indexed.append([y_indexes[j], probas[j]])
    probas_indexed.sort(key=lambda tup: tup[1], reverse=True)
    print "item %d: labels: %s" % (i, probas_indexed[0:5])
    

In [ ]:
feat_importances = []
for i in range(len(X_test.columns)):
    #print "%s: %.3f" % (X_test.columns[i], abo.feature_importances_[i])
    feat_importances.append((X_test.columns[i], gbc.feature_importances_[i]))

feat_importances = sorted(feat_importances, key=lambda x: x[1], reverse=True)
for pair in feat_importances:
    print pair[0], pair[1]

## Multi Layer Perceptron  (scikit) -- only available @v.0.18.dev0 as of 2016-01-25.

In [21]:
# first scale data:
from sklearn.preprocessing import StandardScaler  
scaler = StandardScaler()  
# Don't cheat - fit only on training data
scaler.fit(X_train)  
X_train_scaled = scaler.transform(X_train)  
# apply same transformation to test data
X_test_scaled = scaler.transform(X_test) 

In [33]:
print X_train_scaled.shape
print X_test_scaled.shape

(51670, 770)
(22145, 770)


In [ ]:
from sklearn.neural_network import MLPClassifier

#mlperc = MLPClassifier(algorithm='l-bfgs', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=1)
mlperc = MLPClassifier(hidden_layer_sizes=(15,), verbose=True, random_state=1)
mlperc.fit(X_train_scaled, y_train)

# Neural Network (PyBrain) - first SCALE features!!!!

## UNSCALED DATA:
### ([6], bias=True, hiddenclass=TanhLayer, outclass=SoftmaxLayer, 30 epochs: test error= ~%36.00, Acc=0.6260

### ([20], bias=True, hiddenclass=TanhLayer, outclass=SoftmaxLayer, 30 epochs: test error= %35.45, Acc=0.6429
### ([20], bias=True, hiddenclass=TanhLayer, outclass=SoftmaxLayer, 60 epochs: test error= %35.45, Acc=0.6455
### """ epoch:   90   train error: 35.72%   test error: 36.26% Acc=0.6374

### ([12], bias=True, hiddenclass=TanhLayer, outclass=SoftmaxLayer, epoch:   30   train error: 36.75%   test error: 36.46%, Acc=0.6354

### ([30], bias=True, hiddenclass=TanhLayer, outclass=SoftmaxLayer, epoch:   30   train error: 41.24%   test error: 40.88%, Acc=0.5912
### Smallest test_error: %35.25
#### epoch:   31   train error: 35.54%   test error: 35.47% => Acc=0.6453
#### epoch:   42   train error: 35.53%   test error: 35.35% => Acc=0.6465

### ([24,24], bias=True, hiddenclass=TanhLayer, outclass=SoftmaxLayer, , Acc=0.6397 (epoch 30)
### Smallest test_error: %34.98 epoch:55 => Acc=0.6502

### ([24,24,24], bias=True, hiddenclass=TanhLayer, outclass=SoftmaxLayer,
### Smallest test_error: %35.35 epoch:35/60 => Acc=0.6465

### ([24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24], bias=True, hiddenclass=TanhLayer, outclass=SoftmaxLayer, 
### Smallest epoch:    9   train error: 41.78%   test error: 41.36% => Acc=0.58.64

### ([60], bias=True, hiddenclass=TanhLayer, outclass=SoftmaxLayer, 
### epoch:   29   train error: 35.52%   test error: 35.40%

## SCALED DATA:
### ([4], bias=True, hiddenclass=TanhLayer, outclass=SoftmaxLayer, 50 epochs: test error= ~%XXXX, Acc=0.XXXX


In [34]:
from pybrain.tools.shortcuts import buildNetwork
from pybrain.structure import TanhLayer
from pybrain.structure import SoftmaxLayer
from pybrain.datasets import ClassificationDataSet
from pybrain.supervised.trainers import BackpropTrainer
from pybrain.utilities import percentError

trndata = None
tstdata = None

trndata = ClassificationDataSet(770, 1, nb_classes=12)
for i in range(X_train_scaled.shape[0]):
    #trndata.addSample(X_train[i:i+1].values.tolist()[0], y_train[i:i+1].values.tolist())
    trndata.addSample(X_train_scaled[i:i+1], y_train[i:i+1].values.tolist())
    
tstdata = ClassificationDataSet(770, 1, nb_classes=12)
for i in range(X_test_scaled.shape[0]):
    #tstdata.addSample(X_test[i:i+1].values.tolist()[0], y_test[i:i+1].values.tolist())
    tstdata.addSample(X_test_scaled[i:i+1], y_test[i:i+1].values.tolist())
        
#tstdata, trndata = alldata.splitWithProportion( 0.3 )

trndata._convertToOneOfMany( )
tstdata._convertToOneOfMany( )
 

In [35]:
#net = buildNetwork(150, 100, 50, 12, 1, bias=True, hiddenclass=TanhLayer, outclass=SoftmaxLayer)
#fnn = buildNetwork( trndata.indim, 20, 20, 20, 20, 20, trndata.outdim, bias=True, hiddenclass=TanhLayer, outclass=SoftmaxLayer ) # yields %45.62 test error
fnn = buildNetwork( trndata.indim, 4, trndata.outdim, bias=True, hiddenclass=TanhLayer, outclass=SoftmaxLayer)
                   
#trainer = BackpropTrainer( fnn, dataset=trndata, momentum=0.1, verbose=True, weightdecay=0.01) 
trainer = BackpropTrainer( fnn, dataset=trndata, verbose=True) 

for i in range(50):
    trainer.trainEpochs( 1 )
    trnresult = percentError( trainer.testOnClassData( dataset=trndata ), trndata['class'] )
    tstresult = percentError( trainer.testOnClassData( dataset=tstdata ), tstdata['class'] )

    print "epoch: %4d" % trainer.totalepochs, \
          "  train error: %5.2f%%" % trnresult, \
          "  test error: %5.2f%%" % tstresult   

Total error: 0.0212140951562
epoch:    1   train error: 34.68%   test error: 35.00%
Total error: 0.0195870764483
epoch:    2   train error: 32.88%   test error: 33.40%
Total error: 0.0191875399933
epoch:    3   train error: 32.29%   test error: 32.85%
Total error: 0.0189873041933
epoch:    4   train error: 31.81%   test error: 32.07%
Total error: 0.0188129436134
epoch:    5   train error: 31.68%   test error: 32.34%
Total error: 0.0187421410748
epoch:    6   train error: 31.33%   test error: 31.98%
Total error: 0.0186412671662
epoch:    7   train error: 32.77%   test error: 33.51%
Total error: 0.0185720096949
epoch:    8   train error: 30.94%   test error: 31.75%
Total error: 0.0185287502989
epoch:    9   train error: 32.07%   test error: 32.89%
Total error: 0.0184722163939
epoch:   10   train error: 30.68%   test error: 31.47%
Total error: 0.0184343583148
epoch:   11   train error: 30.55%   test error: 31.42%
Total error: 0.0184013669993
epoch:   12   train error: 30.51%   test error:

In [ ]:
# print some shapes and vars
print "X_train.shape: %s" % str(X_train.shape)
print "y_train.shape: %s" % str(y_train.shape)
print "X_test.shape: %s" % str(X_test.shape)
print "y_test.shape: %s" % str(y_test.shape)

print "X_train_scaled.shape: %s" % str(X_train_scaled.shape)
print "X_test_scaled.shape: %s" % str(X_test_scaled.shape)

print X_train_scaled[0:5]

print "trndata:\n%s" % trndata
print "tstdata:\n%s" % tstdata

print "model:\n%s" % fnn

In [ ]:
y_pred_fnn = trainer.testOnClassData(dataset=tstdata)
conf_matrix_fnn = metrics.confusion_matrix(tstdata["class"], y_pred_fnn)
print conf_matrix_fnn
print sum(sum(conf_matrix_fnn))
metrics.accuracy_score(tstdata["class"], y_pred_fnn)
#tstdata["class"].shape
#trainer.testOnClassData(dataset=tstdata).shape

In [ ]:
for i in range(30):
    trainer.trainEpochs( 1 )
    trnresult = percentError( trainer.testOnClassData(),
                              trndata['class'] )
    tstresult = percentError( trainer.testOnClassData(
           dataset=tstdata ), tstdata['class'] )

    print "epoch: %4d" % trainer.totalepochs, \
          "  train error: %5.2f%%" % trnresult, \
          "  test error: %5.2f%%" % tstresult    